In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
df = pd.read_csv('/content/anonymized_student_data_with_departments_results.csv')

In [ ]:
df

,STUDENT ID,SCHOOL NAME,Biology,Chemistry,Civic Education,Commerce,Economics,English,Further Mathematics,Government,Geography,Literature in English,Mathematics,Physics,Department
0,90673.0,"Barewa College, Zaria",A1,C4,ABS,ABS,ABS,F9,F9,ABS,C6,ABS,F9,F9,Arts
1,90674.0,"Barewa College, Zaria",ABS,ABS,ABS,ABS,ABS,A1,ABS,F9,F9,F9,B3,ABS,Science
2,90675.0,"Barewa College, Zaria",E8,E8,ABS,ABS,ABS,F9,A1,ABS,F9,ABS,A1,C5,Commercial
3,90676.0,"Barewa College, Zaria",C5,B3,ABS,ABS,ABS,F9,F9,ABS,C4,ABS,F9,F9,Commercial
4,90677.0,"Barewa College, Zaria",ABS,ABS,ABS,ABS,ABS,F9,ABS,B3,F9,F9,F9,ABS,Science
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,NaN,"El-Kanemi College, Maiduguri",ABS,ABS,ABS,ABS,ABS,F9,ABS,F9,E8,F9,C4,ABS,Arts
49996,NaN,"El-Kanemi College, Maiduguri",ABS,ABS,ABS,ABS,ABS,E8,ABS,D7,F9,A1,C5,ABS,Commercial
49997,NaN,"El-Kanemi College, Maiduguri",ABS,ABS,ABS,ABS,ABS,A1,ABS,B3,F9,D7,A1,ABS,Science
49998,NaN,"El-Kanemi College, Maiduguri",ABS,ABS,ABS,D7,F9,F9,ABS,D7,ABS,ABS,F9,ABS,Arts


In [ ]:
df.to_csv('teacher_feedback_data.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   STUDENT ID             2959 non-null   float64
 1   SCHOOL NAME            50000 non-null  object 
 2   Biology                50000 non-null  object 
 3   Chemistry              50000 non-null  object 
 4   Civic Education        50000 non-null  object 
 5   Commerce               50000 non-null  object 
 6   Economics              50000 non-null  object 
 7   English                50000 non-null  object 
 8   Further Mathematics    50000 non-null  object 
 9   Government             50000 non-null  object 
 10  Geography              50000 non-null  object 
 11  Literature in English  50000 non-null  object 
 12  Mathematics            50000 non-null  object 
 13  Physics                50000 non-null  object 
 14  Department             50000 non-null  object 
dtypes:

In [ ]:
df.describe()

,STUDENT ID
count,2959.000000
mean,90205.450490
std,35869.604481
min,41282.000000
25%,65371.500000
50%,90136.000000
75%,96325.500000
max,219721.000000


In [ ]:
# Define a mapping for grades
grade_mapping = {
    'A1': 6, 'B2': 5, 'B3': 4, 'C4': 3, 'C5': 2, 'C6': 1,
    'D7': 0, 'E8': 0, 'F9': 0, 'ABS': None
}

# Apply the grade mapping to all relevant columns
subject_columns = [
    'Biology', 'Chemistry', 'Civic Education', 'Commerce',
    'Economics', 'English', 'Further Mathematics',
    'Government', 'Geography', 'Literature in English',
    'Mathematics', 'Physics'
]

# Replace the grades with numerical values according to the mapping
for col in subject_columns:
    df[col] = df[col].map(grade_mapping)

# Display the modified data
df[subject_columns].head()

,Biology,Chemistry,Civic Education,Commerce,Economics,English,Further Mathematics,Government,Geography,Literature in English,Mathematics,Physics
0,6.0,3.0,NaN,NaN,NaN,0.0,0.0,NaN,1.0,NaN,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,6.0,NaN,0.0,0.0,0.0,4.0,NaN
2,0.0,0.0,NaN,NaN,NaN,0.0,6.0,NaN,0.0,NaN,6.0,2.0
3,2.0,4.0,NaN,NaN,NaN,0.0,0.0,NaN,3.0,NaN,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,0.0,NaN,4.0,0.0,0.0,0.0,NaN


In [ ]:
# Define a function to calculate the synthetic JAMB score
def calculate_synthetic_jamb(row, subject_columns):
    # Get the average of the available subject grades (ignoring NaN values)
    valid_grades = row[subject_columns].dropna()
    if len(valid_grades) == 0:
        return None
    average_grade = valid_grades.mean()

    # Scale the average to a score out of 400
    # The maximum average grade is 6, which we scale to 400
    return (average_grade / 6) * 400

# Apply the function to each row to create the synthetic JAMB score
df['Synthetic JAMB Score'] = df.apply(calculate_synthetic_jamb, axis=1, subject_columns=subject_columns)

# Display the first few rows with the synthetic JAMB score
df[['STUDENT ID', 'Department', 'Synthetic JAMB Score']].head()


,STUDENT ID,Department,Synthetic JAMB Score
0,90673.0,Arts,95.238095
1,90674.0,Science,133.333333
2,90675.0,Commercial,133.333333
3,90676.0,Commercial,85.714286
4,90677.0,Science,53.333333


In [ ]:
# Dropping the rows where the synthetic JAMB score is missing
df = df.dropna(subset=['Synthetic JAMB Score'])

# Define the feature columns (subjects + department)
X = df[subject_columns + ['Department']]
y = df['Synthetic JAMB Score']

# Define a preprocessing pipeline:
# - Impute missing values in the subject grades with the mean
# - One-hot encode the 'Department' column
numerical_features = subject_columns
categorical_features = ['Department']

numerical_transformer = SimpleImputer(strategy='mean')
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Use ColumnTransformer to apply the transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the models to test
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Evaluate each model using cross-validation
model_scores = {}
for name, model in models.items():
    # Create a pipeline that combines preprocessing with the model
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

    # Use cross-validation to evaluate the model
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    model_scores[name] = np.mean(np.sqrt(-scores))  # Root Mean Squared Error (RMSE)

# Display the RMSE of each model to the user
model_scores

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['Civic Education']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['Civic Education']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['Civic Education']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['Civic Education']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/pytho

{'Linear Regression': 8.479041604497004,
 'Random Forest': 6.220383915323638,
 'Gradient Boosting': 8.46549642937362}

The Random Forest model performed the best, achieving the lowest RMSE. This suggests that Random Forest would be the most effective model for predicting synthetic JAMB scores from the available data.

In [ ]:
# Create a pipeline for the best model (Random Forest)
best_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', RandomForestRegressor(random_state=42))])

# Train the pipeline on the full training set
best_pipeline.fit(X_train, y_train)

# Evaluate on the test set
from sklearn.metrics import mean_squared_error

# Make predictions on the test set
y_pred = best_pipeline.predict(X_test)

# Calculate the RMSE on the test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

test_rmse


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['Civic Education']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['Civic Education']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


5.959887233220396

In [ ]:
import joblib

# Save the trained pipeline model
model_path = '/content/random_forest_jamb_model.pkl'
joblib.dump(best_pipeline, model_path)

# Provide the model file to the user
model_path

'/content/random_forest_jamb_model.pkl'

In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import streamlit as st
import joblib
import pandas as pd  # Import pandas

# Load the trained model
model = joblib.load('random_forest_jamb_model.pkl')

# Streamlit app
st.title("JAMB Score Prediction")

# Input fields for subject grades and department
st.header("Enter Student's Grades")

subjects = ['Biology', 'Chemistry', 'Civic Education', 'Commerce',
            'Economics', 'English', 'Further Mathematics',
            'Government', 'Geography', 'Literature in English',
            'Mathematics', 'Physics']
grades = {}

for subject in subjects:
    grades[subject] = st.selectbox(f"{subject} Grade", options=['A1', 'B2', 'B3', 'C4', 'C5', 'C6', 'D7', 'E8', 'F9', 'ABS'], index=9)

department = st.selectbox("Department", ['Arts', 'Science', 'Commercial'])

# Convert the input into a DataFrame
input_data = pd.DataFrame([grades])  # Make sure pandas is imported
input_data['Department'] = department

# Map grades to numerical values
grade_mapping = {
    'A1': 6, 'B2': 5, 'B3': 4, 'C4': 3, 'C5': 2, 'C6': 1,
    'D7': 0, 'E8': 0, 'F9': 0, 'ABS': None
}

for col in subjects:
    input_data[col] = input_data[col].map(grade_mapping)

# Predict button
if st.button('Predict JAMB Score'):
    prediction = model.predict(input_data)[0]
    st.success(f"Predicted JAMB Score: {prediction:.2f}")


Writing app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.30.99.224


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.30.99.224:8501

your url is: https://wild-trees-create.loca.lt
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['Civic Education']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:598: UserWarning: Skipping features without any observed values: ['Civic Education']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
  Stopping...
^C
